# Introduction

For this project, I chose to analyze my sleep data (collected using the AutoSleep app on my Apple Watch), along with the activity data that my Apple Watch records. I am very interested in the health benefits of sleep and exercise, and appreciate how they can regulate mood, reduce stress and even play off of one another. This semester has been very stressful for me and I haven't been making adequate sleep and exercise daily priorities. I am hoping that these findings on my own personal data will motivate me to prioritize myself more in the future. After some searching, I was able to find a way for both of the datasets to be exported as .csv files. Apple Health exports data as XML files but using a third-party application solved that problem. 

#### What tables are in the dataset? 
1. AutoSleep Data 
    * the table provided by AutoSleep shows data that was collected using time stamps, as well as heart rate and respiration measurements. 
1. Activity Data 
